# Sequential Conv + Flatten + Dense

## 00. Imports

In [1]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import confusion_matrix

In [2]:
import sys
sys.path.append("../")

In [3]:
from src.activation.relu import ReluLayer
from src.activation.softmax import SoftmaxLayer
from src.layers.dense import DenseLayer
from src.layers.flatten import FlattenLayer
from src.layers.convolutional import ConvLayer2D
from src.model.sequential import SequentialModel
from src.utils.core import convert_categorical2one_hot, convert_prob2categorical
from src.utils.metrics import calculate_accuracy

## 01. Settings

In [4]:
# number of samples in the train data set
N_TRAIN_SAMPLES = 5000
# number of samples in the test data set
N_TEST_SAMPLES = 500
# number of samples in the validation data set
N_VALID_SAMPLES = 500
# number of classes
N_CLASSES = 10
# image size
IMAGE_SIZE = 28

## 02. Build data set

In [5]:
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
print("trainX shape:", trainX.shape)
print("trainY shape:", trainY.shape)
print("testX shape:", testX.shape)
print("testY shape:", testY.shape)

trainX shape: (60000, 28, 28)
trainY shape: (60000,)
testX shape: (10000, 28, 28)
testY shape: (10000,)


In [6]:
X_train = trainX[:N_TRAIN_SAMPLES, :, :]
y_train = trainY[:N_TRAIN_SAMPLES]

X_test = trainX[N_TRAIN_SAMPLES:N_TRAIN_SAMPLES+N_TEST_SAMPLES, :, :]
y_test = trainY[N_TRAIN_SAMPLES:N_TRAIN_SAMPLES+N_TEST_SAMPLES]

X_valid = testX[:N_VALID_SAMPLES, :, :]
y_valid = testY[:N_VALID_SAMPLES]

**NOTE:** We need to change the data format to the shape supported by my implementation.

In [7]:
X_train = np.moveaxis(X_train, 0, -1) / 255
X_train = np.expand_dims(X_train, axis=2)
y_train = convert_categorical2one_hot(y_train).T
X_test = np.moveaxis(X_test, 0, -1) / 255
X_test = np.expand_dims(X_test, axis=2)
y_test = convert_categorical2one_hot(y_test).T
X_valid = np.moveaxis(X_valid, 0, -1) / 255
X_valid = np.expand_dims(X_valid, axis=2)
y_valid = convert_categorical2one_hot(y_valid).T
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (28, 28, 1, 5000)
y_train shape: (10, 5000)
X_test shape: (28, 28, 1, 500)
y_test shape: (10, 500)
X_valid shape: (28, 28, 1, 500)
y_valid shape: (10, 500)


## 03. Build model

In [8]:
layers = [
    ConvLayer2D.initialize(filters=4, kernel_shape=(3, 3, 1)),
    ReluLayer(),
    ConvLayer2D.initialize(filters=4, kernel_shape=(3, 3, 4)),
    ReluLayer(),
    FlattenLayer(),
    DenseLayer.initialize(input_dim=24 * 24 * 4, output_dim=1000),
    ReluLayer(),
    DenseLayer.initialize(input_dim=1000, output_dim=500),
    ReluLayer(),
    DenseLayer.initialize(input_dim=500, output_dim=100),
    ReluLayer(),
    DenseLayer.initialize(input_dim=100, output_dim=25),
    ReluLayer(),
    DenseLayer.initialize(input_dim=25, output_dim=N_CLASSES),
    SoftmaxLayer()
]

model = SequentialModel(layers=layers)

## 04. Train

In [9]:
model.train(
    x_train=X_train, 
    y_train=y_train, 
    x_test=X_test, 
    y_test=y_test, 
    epochs=100, 
    lr=0.005
)

0
Iteration: 00000 - cost: 2.29778 - accuracy: 0.05600
1


KeyboardInterrupt: 

In [48]:
a = np.array([
    [
        [1, 2, 2, 1],
        [3, 4, 0, 0],
        [5, 2, 1, 1],
        [3, 4, 0, 3]
    ],
    [
        [5, 2, 2, 1],
        [3, 4, 3, 0],
        [2, 2, 1, 1],
        [3, 4, 2, 0]
    ],
])

In [49]:
a.shape

(2, 4, 4)

In [50]:
a = np.moveaxis(a, 0, -1)
a = np.expand_dims(a, axis=3)
a = np.moveaxis(a, -1, -2)

In [51]:
a.shape

(4, 4, 1, 2)

In [56]:
a

array([[[[1, 5]],

        [[2, 2]],

        [[2, 2]],

        [[1, 1]]],


       [[[3, 3]],

        [[4, 4]],

        [[0, 3]],

        [[0, 0]]],


       [[[5, 2]],

        [[2, 2]],

        [[1, 1]],

        [[1, 1]]],


       [[[3, 3]],

        [[4, 4]],

        [[0, 2]],

        [[3, 0]]]])

In [52]:
b = np.array([
    [
        [4, 2],
        [5, 3]
    ],
    [
        [5, 3],
        [4, 2]
    ],
])

In [53]:
b = np.moveaxis(b, 0, -1)
b = np.expand_dims(b, axis=3)
b = np.moveaxis(b, -1, -2)

In [54]:
b

array([[[[4, 5]],

        [[2, 3]]],


       [[[5, 4]],

        [[3, 2]]]])

In [55]:
b.shape

(2, 2, 1, 2)

In [58]:
np.amax(a[0:2, 0:2, :, 0], axis=(0, 1)).shape

(1,)